# Create a Web App to Display Market Data from MT5 Using Python

## Add Dash Support to Jupyter

Dash is a library that allows the creation of web apps writing only python code.

Dash uses `flask` under the hood, a well stablished minimalist python web server for the backend and `React` for the frontend. These are great choices that enable developers to create custom components once they got used to all the tools Dash offers.

Here we are talking about MT5 and python but adding interactivity between them may spark some creativity in your mind and encorage you to dig deeper into it.


The instructions for the setup of the different `ipython` enviroments can be found here: https://github.com/plotly/jupyter-dash

The enviroment requirements may vary depending on which `ipython` interface you are using. I am using the __VSCode__ because it requires almost no setup. Others may require `nodejs` installation.

This is the reference to the code editor _extension_ that makes your life easier: ![Jypter extension for VSCode](../images/setup/tutorial_setup_05_vscode_extension.png "Jypter extension for VSCode")

The way we will setup Dash here will allow you to open the app on your browser as well.

In [ ]:
import os
import pprint
import pytz

import MetaTrader5 as mt5
import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests

from dash import no_update
from dash.dependencies import Input, Output, State
from datetime import datetime as dt
from datetime import timedelta
from dotenv import dotenv_values
from pandas.tseries import offsets
from plotly.subplots import make_subplots

from jupyter_dash import JupyterDash

# Globals
pp = pprint.PrettyPrinter(indent=4)
config = {
    **dotenv_values(".env")
}


## What we will build?

In this tutorial, we will learn how we can create interactivity between python and MT5.

We will create a "Tick Watcher", which is an indicator graph to display changes in ticker's data.

For this very basic example, we will:

Create a page with a dropdown menu containing the tickers descriptions. This will receive user's input.

The dropdown menu is populated using the `symbols_get` function we saw in the previous notebook.

Most of the interation is carried by an `Interval` Dash component that changes the `n_intervals` property in fixed time intervals.

We will write a Dash `callback` to listen to these property changes and pull the fresh data from the terminal.

Then we will display the data in the Indicar graph.

## Connecting to MT5 Terminal
Just like in the first part of the tutorial.

In [ ]:
connected = mt5.initialize(
    path=config['MT5_PATH'],
    login=int(config['LOGIN']),
    password=config['PASSWORD'],
    server=config['SERVER'],
    portable=config['PORTABLE'] == 'True'
)
if not connected:
    print("initialize() failed, error code =", mt5.last_error())
    raise Exception('Could not connect')

## List the Symbols

To build our first app, we will need the list of symbols we want to watch.

From the previous notebook, we have the `symbols_get` to help us. I am calling this function and creating a list of dictionaries to store the data.

__Python dictionaries__ are very simple data structures that organize the data into key-values pairs.

Just like in a word dictionary, the word you lookup to is unique (the key) and it points to a definition (the value). If you look at the output of the next code block you will easily understand the concept.

In [ ]:
grouped_symbols = mt5.symbols_get(group="*USD*")
symbols_data = []
for s in grouped_symbols:
    symbols_data.append(dict(value=s.name, label=s.description))
pp.pprint(symbols_data)

We will use this data to create a `select` element for our app later on.

Pay special attention to `dict` keys: `value` and `label`. These are the keys the component expects to find in the list of options.

In the next code block, I just wrapped this piece of code in a function to call it when needed.

In [ ]:
def retrieve_tickers_for_dropdown():
    symbols_data = []
    grouped_symbols = mt5.symbols_get(group="*USD*")
    for s in grouped_symbols:
        symbols_data.append(dict(value=s.name, label=s.description))
    return symbols_data

## Retrieve Tick Changes

Here we are using `copy_ticks_from` to get the last 2 ticks since the last second.

We will use this piece of code to create an Indicator using Plotly.

In [ ]:
tick_data = mt5.copy_ticks_from(
    'EURUSD',
    dt.now() - timedelta(seconds=1),
    2,
    mt5.COPY_TICKS_INFO
)
tick_data

Again, we embed this piece of code in a function to create the Indicator, passing the `symbol` name as the parameter.

The tick data passes the values to the indicator.

The second argument is optional, known in python as "keyword argument". They work like python dictionaries and are used to pass any arbitrary set of parameters to a function. In this case, I will use this to demonstrate how the values change over the time as the `Interval` component triggers the callback.

In [ ]:
def update_indicator(symbol, **kwargs):
    tick_data = mt5.copy_ticks_from(
        symbol,
        dt.now() - timedelta(seconds=1),
        2,
        mt5.COPY_TICKS_INFO
    )
    if len(tick_data) == 0:  # if the market is closed, there will be no data to display.
        tick_data = [(0,0)]  # then we give the function something to plot and prevent errors.

    previous = tick_data[0]
    latest = tick_data[-1]
    fig = go.Figure(
        go.Indicator(
            title = f'Current ask price: ${round(latest[2], 5)}', # you may want to apply some format to this number.
            mode = "number+delta",
            value = latest[1], # bid price is the first item after the time, so index 1.
            number = {
                'prefix': "$",
                'valueformat': '.5f',
            },
            delta = {
                'position': "top",
                'reference': previous[1],
                'relative': True,
                'valueformat':'.3%'
            },
            domain = {'x': [0, 1], 'y': [0, 1]}
        )
    )
    fig.update_layout(
        title=f'Bid Price change: {symbol} {kwargs.get("interval", "")}',
        title_font_size=32
    )
    return fig



If we call this funcion multiple times, we will be able to see the values changes flowing from the terminal to our app. Try it!

In [ ]:
update_indicator('EURUSD')

But running a notebook cell multiple times is not fun. So, let's ask our app to pull the data for us automatically in a defined interval.

## Put it all together

The next code block is the application's entry point.

Bear in mind that this is the "backend" part.

Under the hood, Dash is creating a `flask` application server and storing it in the `app` variable.

I added the `bootstrap` stylesheet (css) file here just to give a better look to the app, fixing the typography and elements positions.

The `className` you see in the html elements refer to the `bootstrap` css classes.

In [ ]:
external_stylesheets = ["https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta3/dist/css/bootstrap.min.css"]

app = JupyterDash(
    __name__, # The __name__ value helps Dash to internally configure paths and other things.
    external_stylesheets=external_stylesheets # To apply good looking defauls styles to our html elements using bootstrap. 
)

Next, we define just one `app.callback`.

All the parameters are in the format: component id -> component property. In simple terms, a Dash callback is a function that "listens" to property changes in the components indicated by the id.

The exemple below:
- This function is triggered when the property `n_intervals` of the component identified by `clock-ticker` changes.
- The `State` is the value of the property held by the component identified by the id `select-symbol` when the callback is triggered.
- The `Output` is the pair id -> property where the result of the callback will be displayed in the user's interface.

In [ ]:
@app.callback(
    Output('indicator', 'figure'),
    Input('clock-ticker', 'n_intervals'),
    State('select-symbol', 'value')
)
def display_indicator(interval, symbol):
    return update_indicator(symbol, interval=interval)

The last part is the "frontend".

The `layout` is the property of the `app` object that describes how the user interface should look like.

Here, python classes represent html elements tags and are nested and styled to create the user interface.

Please, have a look into the [reference documentation.](https://dash.plotly.com/dash-html-components)

In [ ]:
app.layout = html.Div(
    className='container',
    children= [
        html.H1('Your First Tick Watcher'),
        html.H2('MT5 and Python Integration'),
        dcc.Interval(
            id='clock-ticker',
            interval=1*1000, # in milliseconds
            n_intervals=0
        ),
        dcc.Dropdown(
            className='w-50 p-3',
            id='select-symbol',
            options=retrieve_tickers_for_dropdown(), # this calls a function that retrieves a list of dicts contaning labels and values
            value='EURUSD' # the initial selected value
        ),
        dcc.Graph(id='indicator'),
    ]
)

app.run_server(port=8050, mode="inline")

You can open the browser and see the app running there: http://localhost:8050.

# Conclusion

This is just an idea of interation between python and MT5 that use these tools to display fresh market data to end users.

But you can also check the changes in the tick data to make and automate decisions.

The practical applications are endless.